# Week 6: File Ingestion and Schema Validation
### Data Glacier Internship
### Sarah Sindeband
### 10/12/22

In [84]:
# importing libraries
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [71]:
%%time
# inspecting the file
df = pd.read_csv("Airbnb_Ragusa.csv", delimiter=",")

CPU times: total: 156 ms
Wall time: 170 ms


In [72]:
df.head()

,id,listing_url,name,description,picture_url,host_id,host_url,host_name,host_since,host_location,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,137342.0,https://www.airbnb.com/rooms/137342,Villa with pool 4 Pax in Ragusa - Sicily,The house is a typical ancient holiday house r...,https://a0.muscache.com/pictures/869353/b96f65...,671577,https://www.airbnb.com/users/show/671577,Elio,6/6/2011,"Ragusa, Sicily, Italy",...,9/16/2012,6/5/2022,4.80,5.00,4.90,5.00,5.00,4.60,5.00,0.08
1,1762530.0,https://www.airbnb.com/rooms/1762530,Ensuite Room @ModicaOldTownRooms,Convenient central location in the historic ce...,https://a0.muscache.com/pictures/26106238/ee4d...,530213,https://www.airbnb.com/users/show/530213,Andrea,4/23/2011,"Modica, Sicily, Italy",...,6/9/2014,5/17/2022,4.84,4.89,4.89,4.83,4.88,5.00,4.78,0.19
2,3818525.0,https://www.airbnb.com/rooms/3818525,B&B by Francesco,The property offers a double bedroom with bath...,https://a0.muscache.com/pictures/eae112df-8380...,19636276,https://www.airbnb.com/users/show/19636276,Francesco,8/7/2014,"Modica, Sicily, Italy",...,8/14/2014,4/25/2019,4.48,4.89,4.69,4.93,4.89,4.91,4.59,0.47
3,3882447.0,https://www.airbnb.com/rooms/3882447,"Bright flat + terrace, WiFi, AC, Washing machine","Bright , new and cozy apartment on the fifth f...",https://a0.muscache.com/pictures/67284103/369c...,8326692,https://www.airbnb.com/users/show/8326692,Salvo,8/22/2013,"Modica, Sicily, Italy",...,8/23/2014,6/6/2022,4.90,4.96,4.98,4.98,4.99,4.53,4.90,1.90
4,610644.0,https://www.airbnb.com/rooms/610644,A HEAVEN OF PEACE - LA PERLA DI MODICA,<b>The space</b><br />An old house into the r...,https://a0.muscache.com/pictures/58102326-77c4...,3026417,https://www.airbnb.com/users/show/3026417,Michela,7/23/2012,"Bologna, Emilia-Romagna, Italy",...,6/25/2014,8/24/2021,4.64,4.82,4.79,4.84,4.92,4.86,4.58,0.40


The method above for reading the csv file is not an efficient one for creating an automated process for reading files. One limitation is the use of a specific file name. If the file name is different each time, it would be more efficient to use a variable name and assign the variable to the current file name. Another limitation is the specific file type and delimiter. These can change depending on the file, and a program can be written to accomodate those changes.

### Reading the file with YAML

##### Writing the YAML file

In [73]:
%%writefile file.yaml

file_type: csv
dataset_name: Airbnb_Ragusa
file_name: Airbnb_Ragusa
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - id
    - listing_url
    - name
    - description
    - picture_url
    - host_id
    - host_url
    - host_name
    - host_since
    - host_location
    - host_response_time
    - host_response_rate
    - host_acceptance_rate
    - host_is_superhost
    - host_picture_url
    - host_total_listings_count
    - host_has_profile_pic
    - host_identity_verified
    - neighborhood
    - neighborhood_cleansed
    - latitude
    - longitude
    - property_type
    - room_type
    - accommodates
    - bathrooms_text
    - bedrooms
    - beds
    - amenities
    - price
    - minimum_nights
    - maximum_nights
    - number_of_reviews
    - first_review
    - last_review
    - review_scores_rating
    - review_scores_accuracy
    - review_scores_cleanliness
    - review_scores_checkin
    - review_scores_communication
    - review_scores_location
    - review_scores_value
    - reviews_per_month

Overwriting file.yaml


In [74]:
%%writefile yaml_tester.py

# import libraries
import logging                      # releasing log messages form python programs
import os                           # provides functions for creating & removing directories, fetching contents, changing and identifying current directory
import subprocess                   #execute / manage subprocesses: logical collection of activities that exists only within its parent process
import yaml                         # writing configuration files
import pandas as pd
import datetime 
import gc                           # optional garbage collector interface: frees / reclaims blocks of memory that are no longer in use
import re                           # regular expression: specifies a set of strings that matches it

# can also do this for json file
# reading YAML file

def read_config_file(filepath):                         # filepath = filename
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)               # if it can read it --> will safe_load
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting yaml_tester.py


##### Reading the YAML file with the program

In [95]:
# read config file
import yaml_tester as tester
config_data = tester.read_config_file("file.yaml")

# inspecting yaml data
config_data

{'file_type': 'csv',
 'dataset_name': 'Airbnb_Ragusa',
 'file_name': 'Airbnb_Ragusa',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['id',
  'listing_url',
  'name',
  'description',
  'picture_url',
  'host_id',
  'host_url',
  'host_name',
  'host_since',
  'host_location',
  'host_response_time',
  'host_response_rate',
  'host_acceptance_rate',
  'host_is_superhost',
  'host_picture_url',
  'host_total_listings_count',
  'host_has_profile_pic',
  'host_identity_verified',
  'neighborhood',
  'neighborhood_cleansed',
  'latitude',
  'longitude',
  'property_type',
  'room_type',
  'accommodates',
  'bathrooms_text',
  'bedrooms',
  'beds',
  'amenities',
  'price',
  'minimum_nights',
  'maximum_nights',
  'number_of_reviews',
  'first_review',
  'last_review',
  'review_scores_rating',
  'review_scores_accuracy',
  'review_scores_cleanliness',
  'review_scores_checkin',
  'review_scores_communication',
  'review

In [85]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
print(source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

./Airbnb_Ragusa.csv


,id,listing_url,name,description,picture_url,host_id,host_url,host_name,host_since,host_location,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,137342.0,https://www.airbnb.com/rooms/137342,Villa with pool 4 Pax in Ragusa - Sicily,The house is a typical ancient holiday house r...,https://a0.muscache.com/pictures/869353/b96f65...,671577,https://www.airbnb.com/users/show/671577,Elio,6/6/2011,"Ragusa, Sicily, Italy",...,9/16/2012,6/5/2022,4.80,5.00,4.90,5.00,5.00,4.60,5.00,0.08
1,1762530.0,https://www.airbnb.com/rooms/1762530,Ensuite Room @ModicaOldTownRooms,Convenient central location in the historic ce...,https://a0.muscache.com/pictures/26106238/ee4d...,530213,https://www.airbnb.com/users/show/530213,Andrea,4/23/2011,"Modica, Sicily, Italy",...,6/9/2014,5/17/2022,4.84,4.89,4.89,4.83,4.88,5.00,4.78,0.19
2,3818525.0,https://www.airbnb.com/rooms/3818525,B&B by Francesco,The property offers a double bedroom with bath...,https://a0.muscache.com/pictures/eae112df-8380...,19636276,https://www.airbnb.com/users/show/19636276,Francesco,8/7/2014,"Modica, Sicily, Italy",...,8/14/2014,4/25/2019,4.48,4.89,4.69,4.93,4.89,4.91,4.59,0.47
3,3882447.0,https://www.airbnb.com/rooms/3882447,"Bright flat + terrace, WiFi, AC, Washing machine","Bright , new and cozy apartment on the fifth f...",https://a0.muscache.com/pictures/67284103/369c...,8326692,https://www.airbnb.com/users/show/8326692,Salvo,8/22/2013,"Modica, Sicily, Italy",...,8/23/2014,6/6/2022,4.90,4.96,4.98,4.98,4.99,4.53,4.90,1.90
4,610644.0,https://www.airbnb.com/rooms/610644,A HEAVEN OF PEACE - LA PERLA DI MODICA,<b>The space</b><br />An old house into the r...,https://a0.muscache.com/pictures/58102326-77c4...,3026417,https://www.airbnb.com/users/show/3026417,Michela,7/23/2012,"Bologna, Emilia-Romagna, Italy",...,6/25/2014,8/24/2021,4.64,4.82,4.79,4.84,4.92,4.86,4.58,0.40


In [86]:
%%time
df = pd.read_csv(source_file,config_data['inbound_delimiter'])

CPU times: total: 172 ms
Wall time: 164 ms


### Alternative methods for reading the file

##### Modin

In [114]:
import modin.pandas as m_pd
from distributed import Client
client=Client()
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

Perhaps you already have a cluster running?
Hosting the HTTP server on port 63859 instead

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| tornado | 6.2    | 6.2       | 6.1     |
+---------+--------+-----------+---------+


In [115]:
%%time
m_df = m_pd.read_csv(source_file,config_data['inbound_delimiter'])

CPU times: total: 2.56 s
Wall time: 3.78 s


In [89]:
os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray

In [97]:
%%time
m_df = m_pd.read_csv(source_file,config_data['inbound_delimiter'])

CPU times: total: 344 ms
Wall time: 339 ms


### Validating the columns in the file

In [91]:
# validate the header of the file
tester.col_header_val(df, config_data)

column name and column length validation passed


1

In [107]:
for col in df.columns:
    print(col)

id
listing_url
name
description
picture_url
host_id
host_url
host_name
host_since
host_location
host_response_time
host_response_rate
host_acceptance_rate
host_is_superhost
host_picture_url
host_total_listings_count
host_has_profile_pic
host_identity_verified
neighborhood
neighborhood_cleansed
latitude
longitude
property_type
room_type
accommodates
bathrooms_text
bedrooms
beds
amenities
price
minimum_nights
maximum_nights
number_of_reviews
first_review
last_review
review_scores_rating
review_scores_accuracy
review_scores_cleanliness
review_scores_checkin
review_scores_communication
review_scores_location
review_scores_value
reviews_per_month


In [109]:
for col in config_data['columns']:
    print(col)

id
listing_url
name
description
picture_url
host_id
host_url
host_name
host_since
host_location
host_response_time
host_response_rate
host_acceptance_rate
host_is_superhost
host_picture_url
host_total_listings_count
host_has_profile_pic
host_identity_verified
neighborhood
neighborhood_cleansed
latitude
longitude
property_type
room_type
accommodates
bathrooms_text
bedrooms
beds
amenities
price
minimum_nights
maximum_nights
number_of_reviews
first_review
last_review
review_scores_rating
review_scores_accuracy
review_scores_cleanliness
review_scores_checkin
review_scores_communication
review_scores_location
review_scores_value
reviews_per_month


In [113]:
if tester.col_header_val(df, config_data)==0:
    print("Column validation failed")
else:
    print("Column validation passed")

column name and column length validation passed
Column validation passed


In [111]:
# creating a new file from the updated dataframe
df.to_csv("newFile.txt", sep='|',)

In [112]:
# creating gz file
import gzip
import shutil

with open('newFile.txt', 'rb') as f_in:
    with gzip.open('newFile.txt.gz','wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [68]:
print("Summary for Airbnb_Ragusa.csv")
num_cols = len(df.columns)
print("Columns:",num_cols)
num_rows = len(df)
print("Rows: ", num_rows)
size = os.path.getsize('Airbnb_Ragusa.csv')
print("File size: ", size, "Bytes")

Summary for Airbnb_Ragusa.csv
Columns: 43
Rows:  4996
File size:  8630769 Bytes
